In [ ]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

In [2]:
# Read the .csv file
df_gm = pd.read_csv("C:\Language_Projects\Language_Projects\Python\Flagship_1\part-number-datasets-cleaning\data\\raw_datasets\gmR.csv")

<>:2: SyntaxWarning: "\L" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\L"? A raw string is also an option.
<>:2: SyntaxWarning: "\L" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\L"? A raw string is also an option.
C:\Users\Weverson Barbieri\AppData\Local\Temp\ipykernel_10768\1924807895.py:2: SyntaxWarning: "\L" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\L"? A raw string is also an option.
  df_gm = pd.read_csv("C:\Language_Projects\Language_Projects\Python\Flagship_1\part-number-datasets-cleaning\data\\raw_datasets\gmR.csv")


In [3]:
# Check the df info
df_gm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4298 entries, 0 to 4297
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ModType       2886 non-null   object
 1   Hardware      2881 non-null   object
 2   GM CAL №      2834 non-null   object
 3   ID            2448 non-null   object
 4   P/N           2873 non-null   object
 5   Bosch 0261 №  1947 non-null   object
 6   VIN Example   3458 non-null   object
 7   EPC PN        3180 non-null   object
 8   Year          3540 non-null   object
 9   Make & Model  3558 non-null   object
 10  Engine        3317 non-null   object
 11  Prog Info     835 non-null    object
dtypes: object(12)
memory usage: 403.1+ KB


In [4]:
# Remove rows where all cols have null (NaN) values,
# keeping only rows with at least one valid (non-null) entry.
df_gm_not_allNull = df_gm[~df_gm.isnull().all(axis=1)]

In [5]:
# Keep only rows where more than one col has a non-null value.
# This removes rows with only a single non-null entry, ensuring each row has multiple valid data points.
df_gm_more_than_one_not_null = df_gm_not_allNull[df_gm_not_allNull.notnull().sum(axis=1) != 1]

### GM Manufacturer and Module Type DataFrames

General Motors (GM) includes several vehicle brands, and the information related to engine modules may differ between these brands. Therefore, separate DataFrames will be created for each module type. Each DataFrame will be named according to the value in the first column, which represents the module type (`ModType`). This approach ensures that data for each module type is organized and easily accessible for analysis and processing.

In [6]:
def fill_null_rows(df):
    '''
    Iterate over all columns and fill out all null values.

    Parameters: 
        df (DataFrame): main df.

    Returns:
        df: Return a df with the null values filled out
    '''
    for col in df.columns:
        df[col] = df[col].fillna("Not Available")

    return df

In [7]:
# Function to remove the whitespaces
def remove_whitespaces(df):
    # Loop to iterate over all cols
    for col in df.columns:
        # Remove the writespaces
        df[col] = df[col].str.strip()

    return df

In [8]:
# Function to split multiple part numbers originally in the same line to one row each
def split_part_numbers(df, col):

    # Call the function to remove the whitespaces
    df_rm_spaces = remove_whitespaces(df)

    df_copy = df_rm_spaces.copy()

    # Create a list with the items in each row 
    df_copy[col] = df_copy[col].str.split(",")
    # Explode the items in different rows each and keep the info from the other rows
    df_splitted = df_copy.explode(col)
    # Return the df with the exploded part numbers
    return df_splitted

In [9]:
# Fill null values in the 'Hardware' col by propagating the last valid value forward.
# method='ffill' -> copies the value from the previous row into rows with null values.
df_gm_more_than_one_not_null['Hardware'] = df_gm_more_than_one_not_null['Hardware'].fillna(method='ffill')

C:\Users\Weverson Barbieri\AppData\Local\Temp\ipykernel_10768\1009899977.py:3: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_gm_more_than_one_not_null['Hardware'] = df_gm_more_than_one_not_null['Hardware'].fillna(method='ffill')
C:\Users\Weverson Barbieri\AppData\Local\Temp\ipykernel_10768\1009899977.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gm_more_than_one_not_null['Hardware'] = df_gm_more_than_one_not_null['Hardware'].fillna(method='ffill')


In [10]:
# Remove all lines with the repeated cols
repeated_cols_index_list = df_gm_more_than_one_not_null[df_gm_more_than_one_not_null['ModType'] == 'ModType'].index

In [11]:
# Remove all lines with repeated cols
df_gm_rm_repeated_cols = df_gm_more_than_one_not_null.drop(index=repeated_cols_index_list)

In [12]:
# Map all cols to be deleted
col_map = ['ModType', 'ID', 'VIN Example', 'Year', 'Make & Model', 'Engine', 'Prog Info']

In [13]:
# Remove no needed cols according to the map
df_gm_rm_cols = df_gm_rm_repeated_cols.drop(columns=col_map)

In [14]:
# Show the cols
df_gm_rm_cols.columns

Index(['Hardware', 'GM CAL №', 'P/N', 'Bosch 0261 №', 'EPC PN'], dtype='object')

In [15]:
# Map cols will be renamed
rename_mapping_cols = {
    "P/N": "Service №",
    "Bosch 0261 №": "Other PN"
}

In [16]:
# Rename cols from P/N and Bosch 0261 №
df_gm_rn_cols = df_gm_rm_cols.rename(columns=rename_mapping_cols)

In [17]:
# Call the function to fill out null values
df_gm_fill_null_values = fill_null_rows(df_gm_rn_cols)

In [18]:
# Print the cols
df_gm_fill_null_values.columns

Index(['Hardware', 'GM CAL №', 'Service №', 'Other PN', 'EPC PN'], dtype='object')

### Call the function to the explode all items of the cols

In [19]:
df_gm_explo_hdw_col = split_part_numbers(df_gm_fill_null_values, "Hardware")

In [20]:
df_gm_explo_gmCal_col = split_part_numbers(df_gm_explo_hdw_col, "GM CAL №")

In [21]:
df_gm_explo_servN_col = split_part_numbers(df_gm_explo_gmCal_col, "Service №")

In [22]:
df_gm_explo_otherPn_col = split_part_numbers(df_gm_explo_servN_col, "Other PN")

In [23]:
df_gm_explo_epnPn_col = split_part_numbers(df_gm_explo_otherPn_col, "EPC PN")

In [24]:
# Reset the index
df_gm_reset_index = df_gm_explo_epnPn_col.reset_index(drop=True)

In [25]:
# Check the df info
df_gm_reset_index.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4160 entries, 0 to 4159
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Hardware   4160 non-null   object
 1   GM CAL №   4160 non-null   object
 2   Service №  4160 non-null   object
 3   Other PN   4160 non-null   object
 4   EPC PN     4160 non-null   object
dtypes: object(5)
memory usage: 162.6+ KB


## Standardizing Part Number Dataset Columns
Each part number dataset contains columns with different names because they originate from different manufacturers. One of the goals of cleaning these datasets is to create a standard format that allows for the identification of the hardware part number when inputting other part numbers, such as software part numbers or system part numbers.

Since the identification of the hardware part number will be performed by inputting multiple part numbers at once (via a .csv file), it is necessary — except for the hardware column — to rename the other columns to a standard format: `pnNum` (e.g., `pn1`, `pn2`, etc.). This ensures consistency and enables automated matching and processing across datasets from various manufacturers.

In [26]:
def rename_col_names(df):
    '''
    Except the column hardware, the function standardize the names for pnNumber (pn1, pn2).
    Needed due to each part number dataset has different column names, so that 

    Parameters:
    df (DataFrame): main df.
    col_list (list): List of original column names.

    Returns
    df: Return the main df with the renamed cols

    '''

    # List with the col names
    col_name_list = list(df.columns) 
    
    # Dict to append the old names (key): new names (values) 
    new_cols_dict = {}

    # Count the items under the col list
    col_num = len(col_name_list)
    # Loop to iterate over the list
    for col in col_name_list:
        # Condition to define when lower the col name and when rename it
        if col == "Hardware":
            new_cols_dict[col] = col.lower()
        else:
            # Convert the string to number to sum with the stirng pn and
            # append as value to the dict
            new_cols_dict[col] = "pn" + str(col_num)
            # Return the number from string to int
            col_num = int(col_num)
        
        # Subtract one to create the next col name
        num_col = col_num - 1
        col_num = num_col

    # Return the main df with the cols remanes
    return df.rename(columns=new_cols_dict)

In [27]:
# Call the function to rename the cols
df_gm_rn_cols = rename_col_names(df_gm_reset_index)

In [ ]:
# Export to .csv file
df_gm_rn_cols.to_csv("C:\Language_Projects\Language_Projects\Python\Flagship_1\part-number-datasets-cleaning\data\data_cleaned\pn-gm.csv", index=False)

In [28]:
def import_datasets_to_db(df, df_name=str):
    
    '''
    Import the df cleaned to the db on postgreSQL

    Parameters: 
        df (DataFrame): main df to import to the postgreSQL db.
        df_name: string to label the df into the db.

    Returns: 
        Import the df to the db.
    '''
    # Setting up the connection with the PostgreSQL
    dbname="prescreen_diag_data_api"
    user="postgres"
    password="shakey-10"
    host="localhost"
    port="5432"

    # String connection for SQLAlchemy (using psycopg2 as driver)
    engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}")

    return df.to_sql(df_name, engine, if_exists='replace', index=False)

In [29]:
# Call the function to import the df cleaned to the postgreSQLdb
import_datasets_to_db(df_gm_rn_cols, 'part_numbers_gm')

160